In [49]:
from brainda.datasets import Nakanishi2015, Wang2016
from brainda.paradigms import SSVEP
import numpy as np
from sklearn.model_selection import KFold
import tensorflow as tf

In [10]:
# dataset = Nakanishi2015()
dataset = Wang2016()
events = sorted(list(dataset.events.keys()))
freqs = [dataset.get_freq(event) for event in events]
phases = [dataset.get_phase(event) for event in events]
start_pnt = dataset.events[events[0]][1][0]
delay = 0.14 # seconds
channels = ['PZ', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'O1', 'OZ', 'O2']
srate = 250 # Hz
duration = 0.5 # seconds
paradigm = SSVEP(
    srate=srate, 
    channels=channels, 
    intervals=[(start_pnt+delay, start_pnt+delay+duration+0.1)], # more seconds for TDCA 
    events=events)

In [52]:
X, y, meta = paradigm.get_data(
    dataset, 
    subjects=[1], 
    return_concat=False, 
    n_jobs=1, 
    verbose=False)
eeg = np.zeros((40, 6, 9, 150))
y_temp = np.zeros((40, 6))
for i_target, target in enumerate(X.keys()):
    eeg[i_target] = X[target]
eeg = eeg.transpose((1,0,2,3))
X = eeg[...,np.newaxis]
y = y_temp.T
eeg.shape

(6, 40, 9, 150)

In [53]:
X.shape, y.shape,len(freqs),len(phases) # 6 trials x 40 classes
# (240, 9, 150), (240,) --> (6, 40, 9, 150, 1), (6, 40)

((6, 40, 9, 150, 1), (6, 40), 40, 40)

In [ ]:
def EEGNet_SSVEP(nb_classes, Chans, Samples, 
             dropoutRate = 0.3, kernLength = 256, F1 = 96, 
             D = 1, F2 = 96, dropoutType = 'Dropout'):
    """ SSVEP Variant of EEGNet, as used in [1]. 
    Inputs:
        
      nb_classes      : int, number of classes to classify
      Chans, Samples  : number of channels and time points in the EEG data
      dropoutRate     : dropout fraction
      kernLength      : length of temporal convolution in first layer
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. 
      D               : number of spatial filters to learn within each temporal
                        convolution.
      dropoutType     : Either SpatialDropout2D or Dropout, passed as a string.
      
      
    [1]. Waytowich, N. et. al. (2018). Compact Convolutional Neural Networks
    for Classification of Asynchronous Steady-State Visual Evoked Potentials.
    Journal of Neural Engineering vol. 15(6). 
    http://iopscience.iop.org/article/10.1088/1741-2552/aae5d8
    """
    
    if dropoutType == 'SpatialDropout2D':
        dropoutType = tf.keras.layers.SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = tf.keras.layers.Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input1   = tf.keras.Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = tf.keras.layers.Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = tf.keras.layers.BatchNormalization()(block1)
    block1       = tf.keras.layers.DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = tf.keras.constraints.max_norm(1.))(block1)
    block1       = tf.keras.layers.BatchNormalization()(block1)
    block1       = tf.keras.layers.Activation('elu')(block1)
    block1       = tf.keras.layers.AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)
    
    block2       = tf.keras.layers.SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = tf.keras.layers.BatchNormalization()(block2)
    block2       = tf.keras.layers.Activation('elu')(block2)
    block2       = tf.keras.layers.AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)
        
    flatten      = tf.keras.layers.Flatten(name = 'flatten')(block2)
    
    dense        = tf.keras.layers.Dense(nb_classes, name = 'dense')(flatten)
    softmax      = tf.keras.layers.Activation('softmax', name = 'softmax')(dense)
    
    return tf.keras.Model(inputs=input1, outputs=softmax)
num_channels = 9
timepoints_stimulus_duration = 150
# hyperparameters
epochs = 150
batch_size = 100
learning_rate = 1e-3
dropout_rate = 0.5
# build model
kf = KFold(n_splits=6)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index].reshape(-1, *X[train_index].shape[2:]), X[test_index].reshape(-1, *X[test_index].shape[2:])
    y_train, y_test = y[train_index].reshape(-1), y[test_index].reshape(-1)
    print(y_test[:5], y_train[:5])
    print(X_train.shape, y_train.shape)
    print(X_test.shape, y_test.shape)
    model = EEGNet_SSVEP(len(freqs), num_channels, timepoints_stimulus_duration, dropoutRate=dropout_rate)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['acc'])
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size)